# <a href="https://www.pythontutorial.net/advanced-python/python-iterator-vs-iterable/" style="color:Tomato">Python Iterator vs Iterable</a>

Ở bài này ta sẽ học về sự khác nhau giữa iterator và iterable.

### Tables of Contents
* [Iterators](#1)
* [Iterables](#2)
* [Examining the built-in list and list iterator](#3)
* [Python Iterator and Iterable](#4)
* [Separating an iterator from an iterable](#5)
* [Summary](#sum)

## <a class="anchor" id="1">Iterators</a>

<span style="color:DarkOrange">Một **iterator** là một object có implement *iterator protocol*</span>:
- Hàm `__iter__` trả về object đó
- Hàm `__next__` trả về phần tử tiếp theo

Một khi đã duyệt hết các phần tử bằng iterator, sẽ không thể duyệt lại hay duyệt tiếp được nữa.

## <a class="anchor" id="2">Iterables</a>

<span style="color:DarkOrange">Một **iterable** là một object có thể duyệt được</span>.

Một object là một iterable nếu nó implement hàm `__iter__`, và hàm `__iter__` đó trả về một iterator.

## <a class="anchor" id="3">Examining the built-in list and list iterator</a>

Trong Python, list là một iteratable vì nó có hàm `__iter__` trả về một iterator. Ví dụ:

In [1]:
numbers = [1, 2, 3]

number_iterator = numbers.__iter__()
print(type(number_iterator))

<class 'list_iterator'>


Vì class `list_iterator` implement hàm `__iter__`, ta có thể sử dụng hàm built-in `iter` để lấy iterator object:

In [2]:
numbers = [1, 2, 3]
number_iterator = iter(numbers)

Vì `list_iterator` cũng implement hàm `__next__`. ta có thể sử dụng hàm `next` để duyệt list:

In [3]:
numbers = [1, 2, 3]

number_iterator = iter(numbers)

next(number_iterator)
next(number_iterator)
next(number_iterator)

3

Nếu ta gọi hàm `next` một lần nữa thì ta sẽ được `StopIteration` exception.

Thực tế thì <span style="color:DarkOrange">list sẽ chỉ được tạo một lần, còn iterator sẽ được tạo bất kỳ khi nào mà mình cần duyệt list</span>.

## <a class="anchor" id="4">Python Iterator and Iterable</a>

Xét ví dụ sau:

In [4]:
class Colors:
    def __init__(self):
        self.rgb = ['red', 'green', 'blue']
        self.__index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.__index == len(self.rgb):
            raise StopIteration

        # return the next color
        color = self.rgb[self.__index]
        self.__index += 1
        return color

Ở ví dụ này, class `Colors` vừa là iterable vừa là iterator.

Ví dụ sau sử dùng `for` để duyệt các phần tử của class `Colors`:

In [5]:
colors = Colors()

for color in colors:
    print(color)

red
green
blue


Giờ nếu ta gọi `next(colors)`, sẽ bị `StopIteration` exception.

Nếu ta tiếp tục dùng `for`, ta cũng sẽ không nhận được gì thêm, vì iterator giờ đã bị rỗng:

In [6]:
for color in colors:
    print(color)

Muốn duyệt lại thì ta sẽ phải tạo một object `colors` mới. Như vậy sẽ không hiệu quả lắm.

## <a class="anchor" id="5">Separating an iterator from an iterable</a>

Ta sẽ tách iterator ra khỏi iterable như cách mà Python làm với list.

Đầu tiên ta định nghĩa class `Colors`:

In [7]:
class Colors:
    def __init__(self):
        self.rgb = ['red', 'green', 'blue']

    def __len__(self):
        return len(self.rgb)

Tiếp theo ta định nghĩa class `ColorIterator`:

In [8]:
class ColorIterator:
    def __init__(self, colors):
        self.__colors = colors
        self.__index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.__index >= len(self.__colors):
            raise StopIteration

        # return the next color
        color = self.__colors.rgb[self.__index]
        self.__index += 1
        return color

Giờ ta có thể sử dùng `ColorIterator` để duyệt qua object `Colors` như sau:

In [9]:
colors = Colors()
color_iterator = ColorIterator(colors)

for color in color_iterator:
    print(color)

red
green
blue


Muốn duyệt lại object `Colors`, ta chỉ cần tạo lại instance mới của `ColorIterator`.

Nhưng như vậy cũng không hiệu quả lắm. Ta phải tự tạo lại iterator mới bằng tay, và còn phải nhớ tên class `ColorIterator` nữa.

Ta sẽ khắc phục nó như sau: ta cho class `Colors` implement hàm `__iter__`:

In [10]:
class Colors:
    def __init__(self):
        self.rgb = ['red', 'green', 'blue']

    def __len__(self):
        return len(self.rgb)

    def __iter__(self):
        return ColorIterator(self)


Hàm `__inter__` sẽ trả về một instance mới của class `ColorIterator` luôn. Như vậy ta có thể duyệt qua `Colors` object mà không cần phải tự tạo `ColorIterator` object:

In [11]:
colors = Colors()

for color in colors:
    print(color)


red
green
blue


<span style="color:DarkOrange">Vòng lặp `for` sẽ gọi hàm `__iter__` của object `colors` và lấy được một iterator</span>, sau đó dùng iterator này để duyệt các phần tử của `colors`.

Ta có thể đặt class `ColorInterator` bên trong class `Colors` luôn:

In [12]:
class Colors:
    def __init__(self):
        self.rgb = ['red', 'green', 'blue']

    def __len__(self):
        return len(self.rgb)

    def __iter__(self):
        return self.ColorIterator(self)

    class ColorIterator:
        def __init__(self, colors):
            self.__colors = colors
            self.__index = 0

        def __iter__(self):
            return self

        def __next__(self):
            if self.__index >= len(self.__colors):
                raise StopIteration

            # return the next color
            color = self.__colors.rgb[self.__index]
            self.__index += 1
            return color


## <a class="anchor" id="sum" style="color:Violet"> Tổng kết </a>

- Một iterable là một object implement hàm `__iter__` trả về một iterator.
- Một iterator là một object implement hàm `__iter__` trả về chính nó và hàm `__next__` trả về phần tử tiếp theo.
- <span style="color:DarkOrange">Iterators cũng là iterables</span>.